# Calculate Floating Debris Index

A notebook to calculate the Floating Debris Index (FDI; [1]) and apply to a Sentinel 2 netCDF file processed using ACOLITE

[1] Biermann, L., Clewley, D., Martinez-Vicente, V. et al. Finding Plastic Patches in Coastal Waters using Optical Satellite Data. Sci Rep 10, 5364 (2020). https://doi.org/10.1038/s41598-020-62298-z

In [ ]:
import xarray # Used to read and manipulate netCDF
from dask.diagnostics import ProgressBar # Dask is used to process large netCDF without loading all to RAM this shows progress

In [ ]:
# Open scene into xarray dataset
# Specify chunksize so uses dask and doesn't load all data to RAM
s2_data = xarray.open_dataset("S2A_MSI_2018_04_20_11_21_21_T30VWH_L2R.nc", decode_cf=False, chunks=512)

In [ ]:
nir =  s2_data["rhos_833"]
# Try S2A and S2B wavelengths
try:
    red_edge =  s2_data["rhos_740"]
except KeyError:
    red_edge =  s2_data["rhos_739"]
try:
    swir = s2_data["rhos_1614"]
except KeyError:
    swir = s2_data["rhos_1610"]

# Calculate FDI
fdi = nir - red_edge - (swir - red_edge)*1.636
# Set dataset name to FDI
fdi.name = "FDI"

In [ ]:
# Convert to dataset
fdi_xarray = fdi.to_dataset(name="fdi")

In [ ]:
# Write out to netCDF
# Set up attributes
fdi_xarray["fdi"].attrs = {'long_name': 'Floating Debris Index',
                        'standard_name': 'mass_concentration_of_chlorophyll_a_in_sea_water'}

# Export
print("Writing out")
with ProgressBar():
    fdi_xarray.to_netcdf("S2A_MSI_2018_04_20_11_21_21_T30VWH_L2R_fdi.nc", engine="netcdf4", encoding={"fdi" : {"zlib" : True, "complevel" : 4}})